# Python interface for WEOM

## Prerequisites

- Python version 3.8 to 3.12

## Importing the module
- First step is importing the module into your python environment:

In [ ]:
import weompy

The module's base class, `CoreManager`, provides functionality to connect, communicate, and control WEOM.
> 💡 **Hint:** for methods and functions you can use `help()`. Autocomplete is also available.

In [ ]:
help(weompy.CoreManager)

## Connecting to WEOM

* The `CoreManager` class exposes these methods for connection to WEOM:
    * `connectUart(port, baudrate)`
    * `connectUart(port)` - will try to connect to the port with all available baudrates
    * `connectUartAuto()` - will try to connect to all ports with all available baudrates
    * `connectGigeWithID(connectionID)` - will connect to a GigE plugin based on the given connectionID
    * `connectGigeWithDevice(gigeDevice)` - will connect to a GigE plugin based on the given gigeDevice 
> 📝 **Note:** Select the appropriate method based on your plugin
* to disconnect:
    * `disconnect()`

In [ ]:
core = weompy.CoreManager()
try:
    core.connectUartAuto()
    print(f'Connected to port {core.getPortName()}')
except Exception as e:
    print(str(e))

### Methods for controlling WEOM

These methods are used to control the device

* `runMotorFocusCalibration` is used to manually trigger the calibration of the motor focus
* `runNucOffsetUpdate` is used for manually triggering a NUC update
* `resetCore` will restart the device. It will boot into the same mode (MAIN/LOADER) it was restarted from
* `resetToFactoryDefault` will reset the device to default factory settings


In [ ]:
core.runNucOffsetUpdate()

## Property
* to access each individual parameter of WEOM, you can use the so called `Property`
* each property has its own unique identifier (string), with which you can address it
* because the properties are changing based on the connected device, the `CoreManager` class has the following methods to manipulate them:
    * `getPropertyIds()` - returns a list of identifiers for all available properties
    * `isPropertyReadable(property)` - returns `True`, if the property is readable
    * `isPropertyWritable(property)` - returns `True`, if the property is writable
    * `hasProperty(property)` - returns `True`, if the property is readable or writable
    * `getPropertyDescription(property)` - returns the property description
> ⚠️ **Warning:** The methods above can be called without an active connection to the device. The read/write permissions change based on the connected device.
* use these methods for getting/setting property values:
    * `getPropertyValue(property)` - returns the property value, will throw an exception if the property cannot be read from WEOM, or if it failed to validate
    * `setPropertyValue(property, value)` - will try to set `property` to `value`, will throw an exception if it fails


In [ ]:
core.getPropertyIds()

In [ ]:
propertyId = 'NUC_ADAPTIVE_THRESHOLD_CURRENT'
print(core.getPropertyDescription(propertyId))
if core.isPropertyReadable(propertyId):
    try:
        value = core.getPropertyValue(propertyId)
        print(value)
    except Exception as e:
        print(str(e))

In [ ]:
propertyId = 'NUC_ADAPTIVE_THRESHOLD_CURRENT'
if core.isPropertyWritable(propertyId):
    try:
        core.setPropertyValue(propertyId, 2.3)
        print('OK')
    except Exception as e:
        print(str(e))

### Dead pixels (property)
* `DeadPixels` - set of dead pixels and their replacements
* `DeadPixel` - specific dead pixel and its replacement
* `PixelCoordinates` - pixel coordinates [x (1-640), y (1-480)]
> ⚠️ **Warning:**  an attempt to create `PixelCoordinates` with invalid values will throw an exception

In [ ]:
propertyId = 'DEAD_PIXELS_IN_FLASH'
print(core.getPropertyDescription(propertyId))
if core.isPropertyReadable(propertyId):
    try:
        value = core.getPropertyValue(propertyId)
        print(value)
        for dp in value.getDeadPixelsList():
            print(dp)
    except Exception as e:
        print(str(e))

In [ ]:
deadPixels = weompy.DeadPixels()

dp1 = weompy.DeadPixel(1, 5)
deadPixels.insertDeadPixel(dp1)

deadPixels.recomputeReplacements() # create automatic replacements

dp2 = weompy.DeadPixel(23, 12)
dp2.addReplacement(weompy.PixelCoordinates(23, 11)) # manual replacement 1
dp2.addReplacement(weompy.PixelCoordinates(24, 12)) # manual replacement 2

try:
    deadPixels.insertDeadPixel(dp2) # insert pixel with manual replacements
except Exception as e:
    print(str(e))

print(deadPixels)
for dp in deadPixels.getDeadPixelsList():
    print(dp)

propertyId = 'DEAD_PIXELS_CURRENT'
if core.isPropertyWritable(propertyId):
    try:
        core.setPropertyValue(propertyId, deadPixels) # trigger for dead pixel removal is called automatically
        print('OK')
    except Exception as e:
        print(str(e))

## Triggers

They are divided into two types of triggers `common` and `reset`, common triggers work with the properties of the device and reset works with its operation

* `activateCommonTrigger(trigger)`
    * `NUC_OFFSET_UPDATE` - for main, triggers nuc offset update
    * `CLEAN_DP` - for main, clears dead pixels including internal registers (must be called before loading a new set of dead pixels!)
    * `SET_SELECTED_PRESET` - for main, starts setting the selected preset
    * `MOTORFOCUS_CALIBRATION` - for main, starts motorfocus calibration
    * `FRAME_CAPTURE_START` - for main, starts image capture

* `activateResetTriggerAndReconnect(trigger)`
    * `SOFTWARE_RESET` - for main, triggers soft restart and disconnects
    * `RESET_TO_LOADER` - for main, trigger a reboot to loader and disconnect
    * `RESET_TO_FACTORY_DEFAULT` - for main, triggers reset to factory defaults and disconnects

## Image capture

* to get images from WEOM, the `CoreManager` class has these methods:
    * `captureImage()` - returns `Image`
    * `captureImages(count)` - returns `List[Image]`
* frames are represented by the `Image` class with the following methods:
    * `getData()` - returns `List[int]` (specifically `uint16`) with raw image data
    * `save(path)` - saves image to path
    * `load(path)` - static method, loads image from path
    * `getWidth()` - return image width
    * `getHeight()` - returns image height
    * `getPixelValue(pixelCoordinates)` - returns value representation of a pixel, specified by `PixelCoordinates` class

In [ ]:
try:
    image = core.captureImage()
    print(image)
    image.save('image.wti')
    image2 = weompy.Image.load('image.wti')
    print(image.getPixelValue(weompy.PixelCoordinates(1, 1)))
except Exception as e:
    print(str(e))

In [ ]:
try:
    images = core.captureImages(2)
    print(images)
except Exception as e:
    print(str(e))

## Firmware update

* to update WEOM firmware use method `updateFirmare(firmwarePath)` of class `CoreManager`
    * argument `firmwarePath` is used to give the path to the .uwtc firmware file

In [ ]:
try:
    core.updateFirmware('HDMI_0_4_67.uwtc')
    print("OK")
except Exception as e:
    print(str(e))

## Find gigeDevices

* to find GigE devices that you can connect to: use method `findGigeDevices()` of the `CoreManager` class
    * method returns `list[gigeDevice]`, you can use these methods to access info about `gigeDevice`:
        * `getType` returns `USB`, `Network` or `Unknown` based on device type
        * `getMac` returns the MAC address of device if type is `Network`, otherwise ""
        * `getGateway` returns the gateway of device if type is `Network`, otherwise ""
        * `getSubnet` returns the subnet of device if type is `Network`, otherwise ""
        * `getIp` returns the IP address of device if type is `Network`, otherwise ""
        * `getName` returns the name of the device
        * `getSerialNumber` returns the serial number of device
        * `getConnectionID` returns connectionID string, used to connect to device

## Sample to colorize video with our colorization logic
- `colorizeImageDataToArgb(Palette palette, ImageData imageData)` - returns a list in format [A, R, G, B, A, R, G, B, ...], where alpha is 255
- `colorizeImageDataToBgra(Palette palette, ImageData imageData)` - returns a list in format [B, G, R, A, B, G, R, A, ...], where alpha is 255
- `colorizeImageDataToArgb(Palette palette, ImageData imageData, int alpha)` - returns a list in format [A, R, G, B, A, R, G, B, ...], where alpha is configurable
- `colorizeImageDataToBgra(Palette palette, ImageData imageData, int alpha)` - returns a list in format [B, G, R, A, B, G, R, A, ...], where alpha is configurable

In [ ]:
import numpy as np
import cv2
import time

core.startStream(weompy.VideoFormat.POST_COLORING)
palette = core.getPropertyValue("PALETTE_FACTORY_1_CURRENT")

width = 640
height = 480

while True:
    # Capture image and colorize
    image_array = core.getImageDataFromStream()
    colorized = core.colorizeImageDataToBgra(palette, image_array, 255)
    arr = np.frombuffer(colorized, dtype=np.uint8).reshape((height, width, 4))
    cv2.imshow("Colorized Frame", arr)

    # Exit on ESC
    if cv2.waitKey(1) & 0xFF == 27:
        break

cv2.destroyAllWindows()

## Parsing status of device
- `isNucActive()` - returns if camera is running NUC
- `isCameraNotReady()` - returns if camera is ready to respond to commands
- `isCameraInMain()` - returns if camera is in main programme
- `isCameraInLoader()` - returns if camera is in loader
- `isAnyTriggerActive()` - returns if any trigger is currently running on camera

In [ ]:
print(core.isNucActive())
print(core.isCameraNotReady())
print(core.isCameraInMain())
print(core.isCameraInLoader())
print(core.isAnyTriggerActive())

## Reticle
You can change the reticle display mode and position. The display mode has next values: `DISABLED`, `DARK`, `BRIGHT`, `AUTO`, `INVERTED`.
- `DISABLED` - Turns off the reticle display. No aiming cross will be visible.
- `DARK` - Displays the reticle using the lowest color index in the palette, suitable for bright backgrounds.
- `BRIGHT` - Displays the reticle using the highest color index in the palette, suitable for dark backgrounds.
- `AUTO` - Automatically adjusts the reticle color for maximum contrast against the background.
- `INVERTED` - Shows the reticle in the opposite color or intensity to the background for enhanced visibility.

The horizontal range is [-200 : 200]. The vertical range is [-100 : 100]. The code below shows how to set display mode and position for the reticle.

In [ ]:
try:
    core.setPropertyValue('RETICLE_MODE_CURRENT', "BRIGHT")
    core.setPropertyValue('RETICLE_SHIFT_X_AXIS_CURRENT', 100)
    core.setPropertyValue('RETICLE_SHIFT_Y_AXIS_CURRENT', -50)
    print('OK')
except Exception as e:
    print(str(e))